<a href="https://colab.research.google.com/github/nikolasleeb/INFO523_FinalProject/blob/main/FuelPrices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels.api as sm
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
import glob, os

# Attach Data

In [15]:
'''Connects to Google Drive to load in data'''

# Connect to and mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Find the dataset in google drive and assign path to variable called 'Match'
matches = glob.glob('/content/drive/MyDrive/**/MonthlyRetailPrices.xlsx', recursive=True)
print("Found paths:", matches)

# Using 'Match' load in the dataset using the path and skip the first 2 rows
path = matches[0]
prices = pd.read_excel(path, engine='openpyxl', skiprows=2)
prices.head()

'''
# Load dataset from local file for demonstration purposes
consumption = pd.read_excel('ConsumptionData.xlsx', engine='openpyxl')
consumption.head()
'''

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found paths: ['/content/drive/MyDrive/DataMiningFall25/INFO 523 Final Project/Datasets/FuelPrices/MonthlyRetailPrices.xlsx']


,Date,U.S. All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon),U.S. All Grades Conventional Retail Gasoline Prices (Dollars per Gallon),U.S. All Grades Reformulated Retail Gasoline Prices (Dollars per Gallon),U.S. Regular All Formulations Retail Gasoline Prices (Dollars per Gallon),U.S. Regular Conventional Retail Gasoline Prices (Dollars per Gallon),U.S. Regular Reformulated Retail Gasoline Prices (Dollars per Gallon),U.S. Midgrade All Formulations Retail Gasoline Prices (Dollars per Gallon),U.S. Midgrade Conventional Retail Gasoline Prices (Dollars per Gallon),U.S. Midgrade Reformulated Retail Gasoline Prices (Dollars per Gallon),U.S. Premium All Formulations Retail Gasoline Prices (Dollars per Gallon),U.S. Premium Conventional Retail Gasoline Prices (Dollars per Gallon),U.S. Premium Reformulated Retail Gasoline Prices (Dollars per Gallon),U.S. No 2 Diesel Retail Prices (Dollars per Gallon),U.S. No 2 Diesel Ultra Low Sulfur (0-15 ppm) Retail Prices (Dollars per Gallon),U.S. No 2 Diesel Low Sulfur (15-500 ppm) Retail Prices (Dollars per Gallon)
0,1990-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1990-09-15,NaN,NaN,NaN,1.258,1.258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1990-10-15,NaN,NaN,NaN,1.335,1.335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1990-11-15,NaN,NaN,NaN,1.324,1.324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1990-12-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
colors = { "prices": "royalblue" }

# Cleaning and Transforming

In [16]:
''' Verify the columns, datatypes, and number of entries'''
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422 entries, 0 to 421
Data columns (total 16 columns):
 #   Column                                                                           Non-Null Count  Dtype         
---  ------                                                                           --------------  -----         
 0   Date                                                                             422 non-null    datetime64[ns]
 1   U.S. All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon)     390 non-null    float64       
 2   U.S. All Grades Conventional Retail Gasoline Prices (Dollars per Gallon)         370 non-null    float64       
 3   U.S. All Grades Reformulated Retail Gasoline Prices (Dollars per Gallon)         370 non-null    float64       
 4   U.S. Regular All Formulations Retail Gasoline Prices (Dollars per Gallon)        419 non-null    float64       
 5   U.S. Regular Conventional Retail Gasoline Prices (Dollars per Gallon)  

In [17]:
''' Data Cleaning for Prices Data '''

# Keep only needed columns and rename (0,1,13)
prices = prices.iloc[:, [0, 1, 13]]
prices.columns = ["Date", "GasolinePrices", "DieselPrices"]

# Convert Date column to datetime and filter
prices["Date"] = pd.to_datetime(prices["Date"]).dt.to_period("M").dt.to_timestamp()

# Filter to keep data between April 1994 and December 2024
prices = prices[
    (prices["Date"] >= "1995-01-01") &
    (prices["Date"] <= "2021-12-31")
]
# Preview
prices.head(10)

,Date,GasolinePrices,DieselPrices
53,1995-01-01,1.130,1.098
54,1995-02-01,1.120,1.088
55,1995-03-01,1.119,1.088
56,1995-04-01,1.157,1.104
57,1995-05-01,1.225,1.126
58,1995-06-01,1.239,1.120
59,1995-07-01,1.201,1.100
60,1995-08-01,1.170,1.105
61,1995-09-01,1.158,1.119
62,1995-10-01,1.134,1.115


# Summary Stats

In [18]:
prices.describe()

,Date,GasolinePrices,DieselPrices
count,324,324.000000,324.000000
mean,2008-06-16 02:04:26.666666752,2.314812,2.436546
min,1995-01-01 00:00:00,0.962000,0.959000
25%,2001-09-23 12:00:00,1.486750,1.423500
50%,2008-06-16 00:00:00,2.335500,2.507500
75%,2015-03-08 18:00:00,2.928500,3.121500
max,2021-12-01 00:00:00,4.114000,4.703000
std,NaN,0.859399,0.997167


In [19]:
'''' View skewness and kurtosis of prices data '''

columns_to_analyze = prices.columns[1:]  # Exclude the 'Year' column

for column in columns_to_analyze:
    skewness = skew(prices[column].dropna())
    kurt = kurtosis(prices[column].dropna())
    print(f"Column: {column}")
    print(f"  Skewness: {skewness:.4f}")
    print(f"  Kurtosis: {kurt:.4f}\n")

Column: GasolinePrices
  Skewness: 0.1413
  Kurtosis: -1.1255

Column: DieselPrices
  Skewness: 0.1314
  Kurtosis: -1.1753



In [22]:
''' View prices date range and total Months of data '''

print("Earliest date:", prices["Date"].min())
print("Latest date:", prices["Date"].max())
print("Total months of data:", len(prices))

Earliest date: 1995-01-01 00:00:00
Latest date: 2021-12-01 00:00:00
Total months of data: 324


# Gasoline Prices

In [25]:
''' Calculate and print average gasoline prices, standard deviation, and coefficient of variation '''

mean = prices["GasolinePrices"].mean()
std = prices["GasolinePrices"].std()
cv = (std / mean) * 100
print(f"Average monthly Gasoline Prices: {mean:,.2f}")
print(f"Standard deviation: {std:,.2f}")
print(f"Coefficient of variation: {cv:.2f}% \n")

Average monthly Gasoline Prices: 2.31
Standard deviation: 0.86
Coefficient of variation: 37.13% 



In [28]:
''' Create a histogram to visualize the distribution of Gasoline Prices '''

# Create histogram
figr1 = px.histogram(
    prices,
    x="GasolinePrices",
    nbins=15,
    title="Distribution of Gasoline Prices",
    color_discrete_sequence=[colors["prices"]]
)

# Customize layout
figr1.update_layout(
    template="plotly_white",
    title_x=0.5,
    xaxis_title="Gasoline Prices",
    yaxis_title="Frequency",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    hovermode="x unified",
    height=500
)

figr1.update_traces(marker_line_color="black", marker_line_width=1, opacity=1)

figr1.show()

In [32]:
''' Outlier Detection for Gasoline Prices using Z-Score Method and Box Plot '''

# Use an existing column name from the DataFrame (e.g., "Car")
col_name = "GasolinePrices"

# Compute Z-score for the selected column
prices["Z_Gasoline_Prices"] = (
    (prices[col_name] - prices[col_name].mean())
    / prices[col_name].std()
)

# Flag potential outliers (|Z| > 3)
prices_outliers = prices[np.abs(prices["Z_Gasoline_Prices"]) > 3]
print(f"Detected {len(prices_outliers)} potential outliers in '{col_name}'.")
display(prices_outliers[["Date", col_name, "Z_Gasoline_Prices"]])

# visual outlier detection with box plot
figr3 = px.box(
    prices,
    y=col_name,
    title=f"Outlier Detection: {col_name}",
    points="all",
)
figr3.update_layout(template="plotly_white", title_x=0.5, height=500)
figr3.show()

Detected 0 potential outliers in 'GasolinePrices'.


,Date,GasolinePrices,Z_Gasoline_Prices


# Diesel Prices

In [33]:
''' Calculate and print average diesel prices, standard deviation, and coefficient of variation '''

mean = prices["DieselPrices"].mean()
std = prices["DieselPrices"].std()
cv = (std / mean) * 100
print(f"Average monthly Diesel Prices: {mean:,.2f}")
print(f"Standard deviation: {std:,.2f}")
print(f"Coefficient of variation: {cv:.2f}% \n")

Average monthly Diesel Prices: 2.44
Standard deviation: 1.00
Coefficient of variation: 40.93% 



In [34]:
''' Create a histogram to visualize the distribution of Diesel Prices '''

# Create histogram
figr1 = px.histogram(
    prices,
    x="DieselPrices",
    nbins=15,
    title="Distribution of Diesel Prices",
    color_discrete_sequence=[colors["prices"]]
)

# Customize layout
figr1.update_layout(
    template="plotly_white",
    title_x=0.5,
    xaxis_title="Diesel Prices",
    yaxis_title="Frequency",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    hovermode="x unified",
    height=500
)

figr1.update_traces(marker_line_color="black", marker_line_width=1, opacity=1)

figr1.show()

In [35]:
''' Outlier Detection for Diesel Prices using Z-Score Method and Box Plot '''

# Use an existing column name from the DataFrame (e.g., "Car")
col_name = "DieselPrices"

# Compute Z-score for the selected column
prices["Z_Diesel_Prices"] = (
    (prices[col_name] - prices[col_name].mean())
    / prices[col_name].std()
)

# Flag potential outliers (|Z| > 3)
prices_outliers = prices[np.abs(prices["Z_Diesel_Prices"]) > 3]
print(f"Detected {len(prices_outliers)} potential outliers in '{col_name}'.")
display(prices_outliers[["Date", col_name, "Z_Diesel_Prices"]])

# visual outlier detection with box plot
figr3 = px.box(
    prices,
    y=col_name,
    title=f"Outlier Detection: {col_name}",
    points="all",
)
figr3.update_layout(template="plotly_white", title_x=0.5, height=500)
figr3.show()


Detected 0 potential outliers in 'DieselPrices'.


,Date,DieselPrices,Z_Diesel_Prices


# Line Plot

In [36]:
''' create a line graph showing trends over time for each fuel type '''

fig_line = px.line(
    prices,
    x="Date",
    y=["GasolinePrices", "DieselPrices"],
    title="Fuel Prive Trends Over Time",
    labels={"value": "Dollars", "variable": "Fuel Type"},
    color_discrete_sequence=px.colors.qualitative.Set1
)
fig_line.update_layout(
    template="plotly_white",
    title_x=0.5,
    xaxis_title="Year",
    yaxis_title="Dollars",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    hovermode="x unified",
    height=600
)
fig_line.show()